In [2]:
import pandas as pd
import numpy as np
import requests
import csv

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.46 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.95 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.67 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.93 MB/s


In [3]:
# The code was removed by Watson Studio for sharing.

,Area,Latitude,Longitude
0,Klisa,45.302926,19.823393
1,Slana Bara,45.288153,19.828563
2,Vidovdansko Naselje,45.280669,19.832978
3,Salajka,45.270604,19.836438
4,Podbara,45.262004,19.850332


In [4]:
CLIENT_ID = 'R4HPVE2EPLS5EJD03UM5IPWWNMETZWJUEBZCGHZYEZGLBVJN'
CLIENT_SECRET = '0M1EO4BLBQRCDWR2FOX3OH21FJKIF33G34U3DEWJB4LJIA3Q'
VERSION = '20180605' # Foursquare API version


print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: R4HPVE2EPLS5EJD03UM5IPWWNMETZWJUEBZCGHZYEZGLBVJN
CLIENT_SECRET:0M1EO4BLBQRCDWR2FOX3OH21FJKIF33G34U3DEWJB4LJIA3Q


In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
df_venues = getNearbyVenues(names=df_areas['Area'],
                                   latitudes=df_areas['Latitude'],
                                   longitudes=df_areas['Longitude']
                                  )
df_venues.head()

Klisa
Slana Bara
Vidovdansko Naselje
Salajka
Podbara
Stari Grad
Rotkvarija
Banatic
Grbavica
Adamovicevo Naselje
Sajmiste
Detelinara
Avijaticarsko Naselje
Bistrica
Telep
Adice
Liman 1
Liman 2
Liman 3
Liman 4
Jugovicevo
Sajlovo


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Klisa,45.302926,19.823393,Sole Mio ****,45.310429,19.824550,Hotel
1,Klisa,45.302926,19.823393,Okretnica Jedinice,45.302245,19.823976,Bus Station
2,Klisa,45.302926,19.823393,Roda Megamarket,45.305172,19.833761,Supermarket
3,Klisa,45.302926,19.823393,Kaćka petlja,45.306639,19.826250,Road
4,Klisa,45.302926,19.823393,Milka Z,45.301300,19.829960,Hotel


In [7]:
df_area_venue_category =  df_venues.drop(columns=['Area Latitude', 'Area Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude'])
df_area_venue_category

,Area,Venue Category
0,Klisa,Hotel
1,Klisa,Bus Station
2,Klisa,Supermarket
3,Klisa,Road
4,Klisa,Hotel
5,Klisa,Fast Food Restaurant
6,Klisa,Brewery
7,Klisa,Department Store
8,Slana Bara,Indonesian Restaurant
9,Slana Bara,Pool


In [8]:
df_area_venue_category_count = df_area_venue_category.groupby(['Area', 'Venue Category']).size().reset_index(name='Counts')
df_area_venue_category_count

,Area,Venue Category,Counts
0,Adamovicevo Naselje,BBQ Joint,1
1,Adamovicevo Naselje,Bakery,2
2,Adamovicevo Naselje,Bar,1
3,Adamovicevo Naselje,Bistro,1
4,Adamovicevo Naselje,Café,5
5,Adamovicevo Naselje,Coffee Shop,1
6,Adamovicevo Naselje,Cosmetics Shop,2
7,Adamovicevo Naselje,Creperie,1
8,Adamovicevo Naselje,Gym,1
9,Adamovicevo Naselje,Hotel,4


In [9]:
df_area_venue_category_count.loc[df_area_venue_category_count['Venue Category'] == 'Gym']

,Area,Venue Category,Counts
8,Adamovicevo Naselje,Gym,1
27,Avijaticarsko Naselje,Gym,1
45,Banatic,Gym,1
80,Detelinara,Gym,1
99,Grbavica,Gym,1
238,Rotkvarija,Gym,2
261,Sajmiste,Gym,2
307,Stari Grad,Gym,1
324,Telep,Gym,1
